<img src="https://www.bestdesigns.co/uploads/inspiration_images/4350/990__1511457498_404_walmart.png" alt="WALMART LOGO" />

# Walmart : predict weekly sales

## Company's Description 📇

Walmart Inc. is an American multinational retail corporation that operates a chain of hypermarkets, discount department stores, and grocery stores from the United States, headquartered in Bentonville, Arkansas. The company was founded by Sam Walton in 1962.

## Project 🚧

Walmart's marketing service has asked you to build a machine learning model able to estimate the weekly sales in their stores, with the best precision possible on the predictions made. Such a model would help them understand better how the sales are influenced by economic indicators, and might be used to plan future marketing campaigns.

## Goals 🎯

The project can be divided into three steps:

- Part 1 : make an EDA and all the necessary preprocessings to prepare data for machine learning
- Part 2 : train a **linear regression model** (baseline)
- Part 3 : avoid overfitting by training a **regularized regression model**

## Scope of this project 🖼️

For this project, you'll work with a dataset that contains information about weekly sales achieved by different Walmart stores, and other variables such as the unemployment rate or the fuel price, that might be useful for predicting the amount of sales. The dataset has been taken from a Kaggle competition, but we made some changes compared to the original data. Please make sure that you're using **our** custom dataset (available on JULIE). 🤓

## Deliverable 📬

To complete this project, your team should: 

- Create some visualizations
- Train at least one **linear regression model** on the dataset, that predicts the amount of weekly sales as a function of the other variables
- Assess the performances of the model by using a metric that is relevant for regression problems
- Interpret the coefficients of the model to identify what features are important for the prediction
- Train at least one model with **regularization (Lasso or Ridge)** to reduce overfitting


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you: 

### Part 1 : EDA and data preprocessing

Start your project by exploring your dataset : create figures, compute some statistics etc...

Then, you'll have to make some preprocessing on the dataset. You can follow the guidelines from the *preprocessing template*. There will also be some specific transformations to be planned on this dataset, for example on the *Date* column that can't be included as it is in the model. Below are some hints that might help you 🤓

 #### Preprocessing to be planned with pandas

 **Drop lines where target values are missing :**
 - Here, the target variable (Y) corresponds to the column *Weekly_Sales*. One can see above that there are some missing values in this column.
 - We never use imputation techniques on the target : it might create some bias in the predictions !
 - Then, we will just drop the lines in the dataset for which the value in *Weekly_Sales* is missing.
 
**Create usable features from the *Date* column :**
The *Date* column cannot be included as it is in the model. Either you can drop this column, or you will create new columns that contain the following numeric features : 
- *year*
- *month*
- *day*
- *day of week*

**Drop lines containing invalid values or outliers :**
In this project, will be considered as outliers all the numeric features that don't fall within the range : $[\bar{X} - 3\sigma, \bar{X} + 3\sigma]$. This concerns the columns : *Temperature*, *Fuel_price*, *CPI* and *Unemployment*
 


**Target variable/target (Y) that we will try to predict, to separate from the others** : *Weekly_Sales*

 **------------**

 #### Preprocessings to be planned with scikit-learn

 **Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

 - Categorical variables : Store, Holiday_Flag
 - Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek

### Part 2 : Baseline model (linear regression)
Once you've trained a first model, don't forget to assess its performances on the train and test sets. Are you satisfied with the results ?
Besides, it would be interesting to analyze the values of the model's coefficients to know what features are important for the prediction. To do so, the `.coef_` attribute of scikit-learn's LinearRegression class might be useful. Please refer to the following link for more information 😉 https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

### Part 3 : Fight overfitting
In this last part, you'll have to train a **regularized linear regression model**. You'll find below some useful classes in scikit-learn's documentation :
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

**Bonus question**

In regularized regression models, there's a hyperparameter called *the regularization strength* that can be fine-tuned to get the best generalized predictions on a given dataset. This fine-tuning can be done thanks to scikit-learn's GridSearchCV class : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Also, you'll find here some examples of how to use GridSearchCV together with Ridge or Lasso models : https://alfurka.github.io/2018-11-18-grid-search/


### Part 1 : EDA and data preprocessing


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
#pio.renderers.default = "svg" # to be replaced by "iframe" if working on JULIE

In [3]:
# Import des données de ventes Walmart
dataset = pd.read_csv('Walmart_Store_sales.csv')
df = dataset.copy()

In [4]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [4]:
df.describe(include='all')

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [5]:
df.shape

(150, 8)

On a 150 lignes, donc on a des NULL pour toutes les colonnes exceptée la colonne "Store".

La colonne "Weekly Sales" est le target, donc il faut déjà supprimer les lignes qui ont des NULL dans cette colonne.

In [5]:
# Exclusion des lignes avec une target NULL
mask = (df['Weekly_Sales'].isnull() == False)
df = df.loc[mask,:]

La colonne "Date" est en chaîne de caractères, on ne peut donc pas la traiter directement. Je choisis de la supprimer.

In [6]:
df = df.drop(columns='Date')

In [8]:
print("Statistiques des données :")
display(df.describe(include="all"))
print("Proportion de valeurs manquantes :")
display(df.isnull().sum()/df.shape[0])

Statistiques des données :


,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,1.360000e+02,125.000000,121.000000,124.000000,125.000000,122.000000
mean,10.014706,1.249536e+06,0.072000,60.853967,3.316992,178.091144,7.665582
std,6.124614,6.474630e+05,0.259528,18.514432,0.479540,40.243105,1.619428
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,6.050757e+05,0.000000,45.220000,2.838500,131.637000,6.690000
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,15.250000,1.806386e+06,0.000000,75.950000,3.724000,214.878556,8.150000
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000


Proportion de valeurs manquantes :


Store           0.000000
Weekly_Sales    0.000000
Holiday_Flag    0.080882
Temperature     0.110294
Fuel_Price      0.088235
CPI             0.080882
Unemployment    0.102941
dtype: float64

In [7]:
# Visualisation des dépendances par paires
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

Remarques :

1. On peut considérer :
- une colonne catégorielle "Store" (numéro du store) : comme il y a 20 valeurs possibles, un One-Hot Encoder est contestable (trop de colonnes)
- une colonne catégorielle "Holiday_Flag" : 2 valeurs possibles, donc on peut utiliser le OHE sur cette colonne
- 4 colonnes quantitatives : "Temperature", "Fuel_Price", "CPI" et "Unemployment"

2. Certaines distributions de données ne sont pas uniformes notamment sur les 4 dernières colonnes, il faudra traiter des outliers

In [8]:
target_variable = "Weekly_Sales"
features = ["Store","Holiday_Flag","Temperature","Fuel_Price","CPI","Unemployment"]
columns = df.columns

In [9]:
# Créer une instance de SimpleImputer
imputer = SimpleImputer(strategy='median')

# Imputer les valeurs manquantes
df = imputer.fit_transform(df)
df = pd.DataFrame(df, columns=columns)

In [12]:
print("Proportion de valeurs manquantes :")
display(df.isnull().sum()/df.shape[0])

Proportion de valeurs manquantes :


Store           0.0
Weekly_Sales    0.0
Holiday_Flag    0.0
Temperature     0.0
Fuel_Price      0.0
CPI             0.0
Unemployment    0.0
dtype: float64

In [13]:
df.describe(include='all')

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,136.000000,1.360000e+02,136.000000,136.000000,136.000000,136.000000,136.000000
mean,10.014706,1.249536e+06,0.066176,61.007941,3.328816,179.614026,7.646169
std,6.124614,6.474630e+05,0.249509,17.461092,0.459319,38.911400,1.534239
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,6.050757e+05,0.000000,48.072500,2.853750,132.464884,6.833000
50%,10.000000,1.261424e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,15.250000,1.806386e+06,0.000000,75.175000,3.689750,214.756314,8.100750
max,20.000000,2.771397e+06,1.000000,91.650000,4.193000,226.968844,14.313000


In [10]:
# Visualisation des dépendances par paires
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()

In [11]:
# Exclusion des outliers
col_outliers = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for col in col_outliers:
    moy = df[col].mean()
    std = df[col].std()
    mask = ((df[col] > moy-std*3) & (df[col] < moy+std*3))
    count = df[mask][col].shape[0]
    prop = count/df.shape[0]
    print(f"Proportion des {col:12} entre {moy-std*3:6.2f} et {moy+std*3:6.2f} = {prop*100:6.2f} %")
    df = df[mask]

Proportion des Temperature  entre   8.62 et 113.39 = 100.00 %
Proportion des Fuel_Price   entre   1.95 et   4.71 = 100.00 %
Proportion des CPI          entre  62.88 et 296.35 = 100.00 %
Proportion des Unemployment entre   3.04 et  12.25 =  96.32 %


In [12]:
df.describe(include='all')

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,131.000000,1.310000e+02,131.000000,131.000000,131.000000,131.000000,131.000000
mean,9.938931,1.257990e+06,0.061069,60.602977,3.316473,181.581719,7.407718
std,6.228663,6.577463e+05,0.240376,17.453432,0.454986,38.290248,0.939371
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,5.842439e+05,0.000000,47.085000,2.851500,133.361048,6.833000
50%,9.000000,1.366396e+06,0.000000,62.250000,3.451000,196.919506,7.477000
75%,16.000000,1.809576e+06,0.000000,75.045000,3.683000,214.781675,8.059000
max,20.000000,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000


In [13]:
# Visualisation des dépendances par paires
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=800, width = 800)
fig.show()


 - Categorical variables : Store, Holiday_Flag
 - Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek


In [18]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] # on exclut Year, Month, Day, DayOfWeek
categorical_features = ['Store', 'Holiday_Flag']

### Régression linéaire multivariée

Comme on observe aucune corrélation, une régression linéaire univariée ne donnerait pas de bons résultats.

In [27]:
print("Separating labels from features...")
features_list = ["Store","Holiday_Flag","Temperature","Fuel_Price","CPI","Unemployment"]
target_variable = "Weekly_Sales"

X = df.loc[:,features_list]
y = df.loc[:,target_variable]

print("...Done.")
print()

print('y : ')
print(y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

y : 
0    1572117.54
1    1807545.43
2    1244390.03
3    1644470.66
4    1857533.70
Name: Weekly_Sales, dtype: float64

X :
   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
0    6.0           0.0        59.61       3.045  214.777523         6.858
1   13.0           0.0        42.38       3.435  128.616064         7.470
2   11.0           0.0        84.57       3.451  214.556497         7.346
3    6.0           0.0        78.89       2.759  212.412888         7.092
4    4.0           0.0        62.25       2.756  126.160226         7.896


In [131]:
# Split du dataset en train set et test set
print("Dividing into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("...Done.")

Dividing into train and test sets...
...Done.


In [132]:
# Pipeline pour les colonnes numériques
numeric_transformer = Pipeline(
    steps=[
    ('scaler', StandardScaler())
])

In [133]:
# Pipeline pour les colonnes catégorielles
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # suppression de la première colonne
])

In [134]:
# Utilisation de ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
])

In [135]:
# Preprocessings du train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5])
print()

# Preprocessings du test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test)
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment
123    4.0           0.0        84.59       3.469  129.112500         5.644
71     1.0           0.0        62.25       3.308  218.220509         7.866
16    18.0           0.0        21.33       2.788  131.527903         9.202
100   18.0           0.0        69.12       2.906  132.293936         7.477
127    5.0           0.0        62.37       3.451  212.560411         6.768
...Done.
  (0, 0)	1.4527313461302043
  (0, 1)	0.3120120132558174
  (0, 2)	-1.3591041378591802
  (0, 3)	-1.8435819931297643
  (0, 6)	1.0
  (1, 0)	0.12964053098441086
  (1, 1)	-0.04454880778946007
  (1, 2)	0.99011679969577
  (1, 3)	0.5208981939529199
  (2, 0)	-2.2938543730337564
  (2, 1)	-1.1961738198611667
  (2, 2)	-1.295425060675297
  (2, 3)	1.9425658491925544
  (2, 19)	1.0
  (3, 0)	0.5365176079786635
  (3, 1)	-0.9348435286602796
  (3, 2)	-1.275229579458313
  (3, 3)	0.10695454283898527
  (3, 19)	1

In [136]:
# Entraînement du modèle
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print("...Done.")
print(f"Score du modele de regression (entraînement) : {regressor.score(X_train, y_train)}")

Train model...
...Done.
Score du modele de regression (entraînement) : 0.9642745488800373


In [137]:
print(f"Score du modele de regression (test) : {regressor.score(X_test, y_test)}")

Score du modele de regression (test) : 0.9426652828961004


Le modèle de régression linéaire semble bien coller aux données. Est-ce que ce modèle se généralise bien ?

Pour rappel, on a que 131 lignes et 20 stores différents ce qui augmente le nombre de valeurs explicatives. Pour vérifier on fait une validation croisée.

In [138]:
# Validation croisée
from sklearn.model_selection import cross_val_score

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', regressor)
])

scores = cross_val_score(pipeline, X, y, cv=5)

print(f"Scores de validation croisée : {scores}")
print(f"Score moyen de validation croisée : {scores.mean()}")

Scores de validation croisée : [0.87609122 0.96610448 0.90879888 0.95868578 0.96698561]
Score moyen de validation croisée : 0.9353331939677293


Quels sont les coefficients qui sont les plus importants dans la prédiction ?

Pour cela, on utilise l'attribut "coef_" de notre modèle.

In [46]:
df_sans_target = df.drop(columns='Weekly_Sales')
df_encoded = pd.get_dummies(df_sans_target, columns=['Store', 'Holiday_Flag'], drop_first=True)
features_encoded = df_encoded.columns
coefficients = regressor.coef_
features_coefficients = list(zip(features_encoded, coefficients))
sorted_features_coefficients = sorted(features_coefficients, key=lambda x: abs(x[1]), reverse=True)
for feature, coef in sorted_features_coefficients:
    print(f"{feature}: {coef}")

Store_5.0: -1324688.7204910018
Store_3.0: -1201114.912221905
Store_9.0: -1187461.3374649526
Store_16.0: -1071084.7118245228
Store_7.0: -917386.7444140163
Store_4.0: 730281.1441596273
Store_8.0: -721373.6862888015
Store_15.0: -661548.7413584165
Store_10.0: 654474.3753589629
Store_14.0: 652166.5388862031
Store_13.0: 600656.251546742
Store_17.0: -534712.9346631765
Store_20.0: 404108.7597168501
Store_2.0: 389778.8087162992
Store_18.0: -133217.38215024973
CPI: 93220.20745137187
Store_19.0: 80744.01713000954
Unemployment: -68449.69333257125
Store_6.0: -62634.31683278083
Store_11.0: 59194.60388437854
Fuel_Price: -50671.85619741682
Temperature: -26700.69158572136
Holiday_Flag_1.0: -23621.06346418885


## Régularisation Ridge

In [141]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

# Créer et entraîner le modèle Ridge
ridge_regressor = Ridge(alpha=1.0)
ridge_regressor.fit(X_train, y_train)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ridge_regressor)
])

# Effectuer la validation croisée
ridge_scores = cross_val_score(pipeline, X, y, cv=5)

print(f"Scores de validation croisée Ridge : {ridge_scores}")
print(f"Score moyen de validation croisée Ridge : {ridge_scores.mean()}")

Scores de validation croisée Ridge : [0.82852465 0.91790302 0.83141373 0.93807703 0.96140636]
Score moyen de validation croisée Ridge : 0.8954649564315522
